In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgbm
from xgboost import plot_importance
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_supplement.csv', 'train.csv', 'test.csv', 'sample_submission.csv', 'train_sample.csv']


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
is_valid = False

path = '../input/'

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train = pd.read_csv(path+"train.csv", skiprows=range(1,131886954), nrows=53016937, usecols=train_columns, dtype=dtypes)
test = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)


CPU times: user 2min 16s, sys: 9.79 s, total: 2min 26s
Wall time: 2min 23s


In [ ]:
test.head()

In [3]:
Y=train.is_attributed
train.drop('is_attributed',axis=1,inplace=True)

In [4]:
def timeFeatures(df):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df['click_time'])
    df['dow']      = df['datetime'].dt.dayofweek
    df["hour"]      = df["datetime"].dt.hour
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['datetime'], axis=1, inplace=True)
    return df


In [5]:
test=timeFeatures(test)
train=timeFeatures(train)

In [6]:
nrow_train = train.shape[0]
merge = pd.concat([train, test])
gc.collect()

247

In [7]:
%%time
ip_count = merge.groupby(['ip','dow','hour'])['channel'].count().reset_index()
ip_count.columns = ['ip','dow','hour','clicks_by_ip_dow_hour']
merge = pd.merge(merge, ip_count, on=['ip','dow','hour'], how='left', sort=False)
merge['clicks_by_ip_dow_hour'] = merge['clicks_by_ip_dow_hour'].astype('uint16')
del ip_count
gc.collect()

CPU times: user 47.5 s, sys: 31.8 s, total: 1min 19s
Wall time: 1min 2s


In [8]:
%%time

# Count the number of clicks by ip
ip_count = merge.groupby(['ip'])['channel'].count().reset_index()
ip_count.columns = ['ip', 'clicks_by_ip']
merge = pd.merge(merge, ip_count, on='ip', how='left', sort=False)
merge['clicks_by_ip'] = merge['clicks_by_ip'].astype('uint16')
del ip_count
gc.collect()

ip_count = merge.groupby(['ip','app'])['channel'].count().reset_index()
ip_count.columns = ['ip','app', 'clicks_by_ip_app']
merge = pd.merge(merge, ip_count, on=['ip','app'], how='left', sort=False)
merge['clicks_by_ip_app'] = merge['clicks_by_ip_app'].astype('uint16')
del ip_count
gc.collect()

ip_count = merge.groupby(['ip','os'])['channel'].count().reset_index()
ip_count.columns = ['ip','os', 'clicks_by_ip_os']
merge = pd.merge(merge, ip_count, on=['ip','os'], how='left', sort=False)
merge['clicks_by_ip_os'] = merge['clicks_by_ip_os'].astype('uint16')
del ip_count
#merge.drop('ip', axis=1, inplace=True)
gc.collect()


CPU times: user 3min 8s, sys: 2min 45s, total: 5min 53s
Wall time: 3min 43s


In [9]:
%%time

# Count the number of os by ip
ip_count = merge.groupby(['channel'])['os'].count().reset_index()
ip_count.columns = ['channel', 'clicks_by_channel']
merge = pd.merge(merge, ip_count, on='channel', how='left', sort=False)
merge['clicks_by_channel'] = merge['clicks_by_channel'].astype('uint16')
#merge.drop('channel', axis=1, inplace=True)
del ip_count
print('channel done')

ip_count = merge.groupby(['os'])['channel'].count().reset_index()
ip_count.columns = ['os', 'clicks_by_os']
merge = pd.merge(merge, ip_count, on='os', how='left', sort=False)
merge['clicks_by_os'] = merge['clicks_by_os'].astype('uint16')
#merge.drop('os', axis=1, inplace=True)
del ip_count
print('os done')
gc.collect()

ip_count = merge.groupby(['app'])['channel'].count().reset_index()
ip_count.columns = ['app', 'clicks_by_app']
merge = pd.merge(merge, ip_count, on='app', how='left', sort=False)
merge['clicks_by_app'] = merge['clicks_by_app'].astype('uint16')
#merge.drop('app', axis=1, inplace=True)
del ip_count

print('app done')

train = merge[:nrow_train]
test = merge[nrow_train:]

#del ip_count,merge
#gc.collect()

channel done
os done
app done
CPU times: user 1min 38s, sys: 3min 20s, total: 4min 59s
Wall time: 2min 5s


In [10]:
del merge
gc.collect()

42

In [12]:
train.head()

,ip,app,device,os,channel,click_time,dow,hour,clicks_by_ip_dow_hour,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os,clicks_by_channel,clicks_by_os,clicks_by_app
0,201143,11,1,13,487,2017-11-09 00:00:00,3,0,70,973,56,198,22788,40127,48238
1,34684,2,1,13,469,2017-11-09 00:00:00,3,0,54,1115,68,143,20075,40127,59188
2,207368,26,1,19,477,2017-11-09 00:00:00,3,0,101,1079,27,319,35638,29757,2827
3,110176,18,1,8,121,2017-11-09 00:00:00,3,0,146,2211,214,320,65389,193,11842
4,109644,12,1,19,265,2017-11-09 00:00:00,3,0,393,11980,1423,3032,17596,29757,4903


In [45]:
train['click_time'] = pd.to_datetime(train['click_time'])
test['click_time'] = pd.to_datetime(test['click_time'])

In [47]:
def get_deltatime(df):
    GROUP_BY_NEXT_CLICKS = [
        {'groupby': ['ip']},
        {'groupby': ['ip', 'app']},
        {'groupby': ['ip', 'channel']},
        {'groupby': ['ip', 'os']},
    ]

    # Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:

        # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    

        # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']

        # Run calculation
        print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
        df[new_feature] = df.groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds
        
    df.drop(['click_time'],axis=1,inplace=True)
    return df

In [46]:
gc.collect()

0

In [16]:
test.reset_index(inplace=True,drop=True)


In [18]:
test.head()

,ip,app,device,os,channel,click_time,dow,hour,clicks_by_ip_dow_hour,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os,clicks_by_channel,clicks_by_os,clicks_by_app
0,5744,9,1,3,107,2017-11-10 04:00:00,4,4,34,469,87,6,40721,26647,42174
1,119901,9,1,3,466,2017-11-10 04:00:00,4,4,403,7918,1089,164,21138,26647,42174
2,72287,21,1,19,128,2017-11-10 04:00:00,4,4,229,5719,417,1030,22225,29757,32857
3,78477,15,1,13,111,2017-11-10 04:00:00,4,4,239,4513,193,995,31367,40127,54851
4,123080,12,1,13,328,2017-11-10 04:00:00,4,4,60,583,79,158,30346,40127,4903


In [49]:
x_valid=train[:52486768]
y_valid=Y[:52486768]
x_train=train[52486768:]
y_train=Y[52486768:]

In [50]:
x_train.shape,x_valid.shape,y_train.shape,y_valid.shape

((530169, 15), (52486768, 15), (530169,), (52486768,))

In [ ]:
%%time
x_train=get_deltatime(x_train)
print("nextclick for x_train is done")
x_valid=get_deltatime(x_valid)

>> Grouping by ['ip'], and saving time to next click in: ip_nextClick


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


>> Grouping by ['ip', 'app'], and saving time to next click in: ip_app_nextClick
>> Grouping by ['ip', 'channel'], and saving time to next click in: ip_channel_nextClick
>> Grouping by ['ip', 'os'], and saving time to next click in: ip_os_nextClick


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


>> Grouping by ['ip'], and saving time to next click in: ip_nextClick


In [ ]:
x_train.drop(['ip','click_time'],axis=1,inplace=True)
x_valid.drop(['ip','click_time'],axis=1,inplace=True)
gc.collect()

In [ ]:
    
dtrain = lgbm.Dataset(x_train, label=y_train)
dvalid = lgbm.Dataset(x_valid, label=y_valid)

In [ ]:
gc.collect()

In [ ]:

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves':60,  
    'max_depth': 6,
    'scale_pos_weight': 99.7
    }


evals_results = {}

print("Training the model...")

lgb_model = lgbm.train(params, dtrain,valid_sets=[dtrain, dvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=500)

In [ ]:
print('Feature names:', lgb_model.feature_name())
print('Feature importances:', list(lgb_model.feature_importance()))


In [ ]:
submit = pd.read_csv(path_test, dtype='int', usecols=['click_id'])
submit['is_attributed'] = lgb_model.predict(test[predictors], num_iteration=lgb_model.best_iteration)
submit.to_csv('submission.csv', index=False)


In [ ]:

del train
del val
gc.collect()
